In [ ]:
from __future__ import annotations

from pathlib import Path
import sys
from contextlib import contextmanager
from typing import TYPE_CHECKING

from sotodlib.workflows import get_wafer_offset
from toast.scripts import toast_ground_schedule

if TYPE_CHECKING:
    from typing import Callable

In [ ]:
# helpers to run any python function intended for cli

@contextmanager
def tee_stdout(buffer):
    """Similar to redirect_stdout, but also prints to the original stdout."""
    cur_stdout = sys.stdout
    class TeeStdout:
        def __init__(self, buffer):
            self.buffer = buffer

        def write(self, data):
            self.buffer.write(data)
            cur_stdout.write(data)

        def flush(self):
            self.buffer.flush()
            cur_stdout.flush()

    sys.stdout = TeeStdout(buffer)
    try:
        yield
    finally:
        sys.stdout = cur_stdout


@contextmanager
def tee_stderr(buffer):
    """Similar to redirect_stderr, but also prints to the original stderr."""
    cur_stderr = sys.stderr
    class TeeStderr:
        def __init__(self, buffer):
            self.buffer = buffer

        def write(self, data):
            self.buffer.write(data)
            cur_stderr.write(data)

        def flush(self):
            self.buffer.flush()
            cur_stderr.flush()

    sys.stderr = TeeStderr(buffer)
    try:
        yield
    finally:
        sys.stderr = cur_stderr


def run_cli(func: Callable[[], None], args: list[str]) -> tuple[str, str]:
    """Run any function that is intended to be run from the command line.

    For example, if you have an entry point `module.sub:main` that you can run
    with `python -m module.sub arg1 arg2 ...`, you can run it with this function like so:

    >>> stdout, stderr = run_cli(module.sub.main, ['arg1', 'arg2', ...])
    """
    from unittest.mock import patch
    import io

    f_out = io.StringIO()
    f_err = io.StringIO()
    with tee_stdout(f_out), tee_stderr(f_err):
        # https://stackoverflow.com/a/27765993/5769446
        with patch.object(sys, 'argv', [func.__name__] + args):
            func()
    return f_out.getvalue(), f_err.getvalue()


In [ ]:
def get_wafer_offset_func(
    tube: str,
) -> tuple[float, float, float]:
    print(f"Running stage 1 for {tube}", "Offsets are", sep='\n')
    stdout, _ = run_cli(get_wafer_offset.main, ['--tube_slots', tube])
    offset_az, offset_el, tube_radius, _, _, _ = stdout.split()
    return float(offset_az), float(offset_el), float(tube_radius)

In [ ]:
def toast_ground_schedule_func(
    path: Path,
    offset_az: float,
    offset_el: float,
    tube_radius: float,
    tele: str = "LAT",
    start: str = "2023-06-08 00:00:00",
    stop: str = "2023-06-09 00:00:00",
    sso_name: str = "Jupiter",
):
    _, _ = run_cli(
    toast_ground_schedule.main,
        [
            "--site-lat",
            "-22.958064",
            "--site-lon",
            "-67.786222",
            "--site-alt",
            "5200",
            "--site-name",
            "ATACAMA",
            "--telescope",
            tele,
            "--start",
            start,
            "--stop",
            stop,
            "--boresight-offset-az-deg",
            str(offset_az),
            "--boresight-offset-el-deg",
            str(offset_el),
            "--patch",
            f"{sso_name},SSO,1,{tube_radius}",
            "--out",
            str(path),
        ],
    )

In [ ]:
def stage1(
    tube: str,
    tele: str = "LAT",
    start: str = "2023-06-08 00:00:00",
    stop: str = "2023-06-09 00:00:00",
    sso_name: str = "Jupiter",
):
    offset_az, offset_el, tube_radius = get_wafer_offset_func(tube)

    schedule_file = Path("schedules") / f"schedule_{tube}_{sso_name}.txt"
    toast_ground_schedule_func(
        schedule_file,
        offset_az,
        offset_el,
        tube_radius,
        tele=tele,
        start=start,
        stop=stop,
        sso_name=sso_name,
    )

In [ ]:
stage1("c1", sso_name="Jupiter")